In [ ]:
import pandas as pd
import numpy as np
from scipy.stats.mstats import winsorize
import os

# 1. 경로 설정
raw_path = '../../0_data/raw/TS2000_RAW.xlsx'
processed_path = 'preprocessed_data.csv'  # 현재 노트북 디렉토리에 저장

# 2. 데이터 불러오기
df = pd.read_excel(raw_path)

# 3. 결측치 처리
threshold = 0.8
cols_to_impute = df.columns[df.notna().mean() >= threshold]
cols_to_drop_by_year = df.columns[df.notna().mean() < threshold]

# 중앙값으로 결측치 대체
for col in cols_to_impute:
    if df[col].isna().sum() > 0:
        median_val = df[col].median()
        df[col] = df[col].fillna(median_val)

# 4. '회계년도' 기준 결측치 행 제거
year_col = '회계년도'
if year_col in df.columns:
    print(f"🗓️ '{year_col}' 기준으로 결측치 포함된 연도 제거 중...")
    for col in cols_to_drop_by_year:
        missing_years = df[df[col].isna()][year_col].unique()
        df = df[~df[year_col].isin(missing_years)]
else:
    print(f"⚠️ '{year_col}' 컬럼이 존재하지 않아 연도 기준 삭제를 건너뜁니다.")

# 5. 윈저라이징 (상하위 0.5%)
numeric_cols = df.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    mask = df[col].notna()
    if mask.sum() > 0:
        df.loc[mask, col] = winsorize(df.loc[mask, col], limits=[0.005, 0.005])

# 6. 저장 (디렉토리가 있으면 생성, 없으면 건너뜀)
dir_name = os.path.dirname(processed_path)
if dir_name:  # 비어 있지 않은 경우에만 디렉토리 생성
    os.makedirs(dir_name, exist_ok=True)

df.to_csv(processed_path, index=False)
print("✅ 전처리 완료. 저장 경로:", os.path.abspath(processed_path))


🗓️ '회계년도' 기준으로 결측치 포함된 연도 제거 중...


FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: ''